In [23]:
import yaml
import os
# !pip install e3nn
from e3nn import o3

In [24]:
def load_config(config_file_path):
    with open(config_file_path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

def save_config(config, config_file_path):
    with open(config_file_path, 'w') as stream:
        try:
            yaml.dump(config, stream, default_flow_style=False)
        except yaml.YAMLError as exc:
            print(exc)

def update_config(config, **kwargs):
    for key, value in kwargs.items():
        config[key] = value
    return config

def irreps(lvalue,num_features=32,even=False):
    return(str(o3.Irreps(
                    [
                        (num_features, (l, p))
                        for p in ((1, -1) if not even else (1,))
                        for l in range(lvalue + 1)
                    ]
                )))

In [48]:
example_config_file_path="old_configs/aspirin.yaml"
num_features=32
max_epochs=200
root="results/aspirin3"
out_dir_configs="aspirin_configs"
feature_irreps_hidden=""

if not os.path.exists(out_dir_configs):
    os.makedirs(out_dir_configs)

!rm -rf {out_dir_configs}
!mkdir {out_dir_configs}

n_val=50
for n_train in [900]:
    for inv_layers in [0,1]:
        for lmax in [1,2,3]:

            config = load_config(example_config_file_path)

            num_layers=4-inv_layers
            layer_irreps=[irreps(lvalue=lmax,num_features=num_features,even=False) for _ in range(num_layers)]
            layer_irreps+=[irreps(lvalue=0,num_features=num_features,even=True) for _ in range(inv_layers)]
            
            assert len(layer_irreps)==4
            
            run_name=f"ntrain_{n_train}_lmax_{lmax}_invariant_{inv_layers}_epochs_{max_epochs}"

            config = update_config(config, n_val=n_val,num_layers=num_layers, max_epochs=max_epochs, root=root, run_name=run_name,layer_irreps=layer_irreps, n_train=n_train, feature_irreps_hidden=feature_irreps_hidden)
            save_config(config, f"{out_dir_configs}/{run_name}.yaml")

In [46]:
example_config_file_path="old_configs/aspirin.yaml"
num_features=[32,32,128,128]
max_epochs=200
root="results/aspirin_4"
out_dir_configs="aspirin_configs"
n_val=50
l_values=[3,3,3,0]

num_layers=len(l_values)
layer_irreps=",".join([irreps(lvalue,num_features=num_features[i],even=(lvalue==0)) for i,lvalue in enumerate(l_values)])
feature_irreps_hidden=""

# if not os.path.exists(out_dir_configs):
#     os.makedirs(out_dir_configs)

# !rm -rf {out_dir_configs}
# !mkdir {out_dir_configs}

for n_train in [900]:

    config = load_config(example_config_file_path)

    run_name=f"ntrain_{n_train}_irreps_{''.join([str(l) for l in l_values])}_epochs_{max_epochs}"

    config = update_config(config, n_val=n_val,num_layers=num_layers, max_epochs=max_epochs, root=root, run_name=run_name,layer_irreps=layer_irreps, n_train=n_train, feature_irreps_hidden=feature_irreps_hidden)
    save_config(config, f"{out_dir_configs}/{run_name}.yaml")